In [29]:
import pandas as pd
import seaborn as sns
import bs4
import requests
import os
from scipy.misc import imresize
from bleedml.classifiers import CascadeForest
from sklearn.ensemble import RandomForestClassifier
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
def get_image_links(search, limit=100):
    link = 'https://www.google.com/search?q={search}&oq={search}&start={start}'
    start = 0
    links = []
    while len(links) < limit:
        print(start)
        page = requests.get(link.format(search=search, start=0))
        if page.status_code == 200:
            soup = bs4.BeautifulSoup(page.text, 'html5lib')
            newlinks = [i.get('src') for i in soup.find_all('img')]
            start += len(newlinks)
            links.extend(newlinks)
        else:
            print('error in link')
    return links

In [ ]:
links = get_image_links('wheat')
with open('wheatlinks', 'w') as fl:
    fl.write('\n'.join(links))

In [6]:
! mkdir ../images/wheat/
! cd ../images/wheat/ && wget -i ../../notebooks/wheatlinks

mkdir: cannot create directory ‘../images/wheat/’: File exists
--2017-03-20 18:55:22--  https://encrypted-tbn1.gstatic.com/images?q=tbn:ANd9GcRUbTwSA164YCbHmfZUJ43BqoRzmc1h9HgfC9wWDUrNIxjm28SD288QoA
Resolving encrypted-tbn1.gstatic.com... 216.58.220.206, 2404:6800:4002:804::200e
Connecting to encrypted-tbn1.gstatic.com|216.58.220.206|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4538 (4.4K) [image/jpeg]
Saving to: ‘images?q=tbn:ANd9GcRUbTwSA164YCbHmfZUJ43BqoRzmc1h9HgfC9wWDUrNIxjm28SD288QoA’

images?q=tbn:ANd9Gc 100%[===================>]   4.43K  --.-KB/s    in 0s      

2017-03-20 18:55:22 (11.1 MB/s) - ‘images?q=tbn:ANd9GcRUbTwSA164YCbHmfZUJ43BqoRzmc1h9HgfC9wWDUrNIxjm28SD288QoA’ saved [4538/4538]

--2017-03-20 18:55:22--  https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT0tXWQETNFmy7yu8gK-IxpkP6s8b3msXKraveOXxL_ZdM62Qe7_K7TTSUM
Resolving encrypted-tbn0.gstatic.com... 216.58.220.206, 2404:6800:4002:804::200e
Connecting to encrypted-tbn0.gstatic.com|216

In [7]:
links = get_image_links('barley')
with open('barleylinks', 'w') as fl:
    fl.write('\n'.join(links))

0
12
24
35
46
58
69
80
92


In [8]:
! mkdir ../images/barley/
! cd ../images/barley/ && wget -i ../../notebooks/barleylinks

mkdir: cannot create directory ‘../images/wheat/’: File exists
--2017-03-20 18:56:19--  https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT7S5ICg0CFXhBnCraxvE00bujuyKEBNX0oseeqbymdVA3lmO_fp3oIFg
Resolving encrypted-tbn0.gstatic.com... 216.58.220.206, 2404:6800:4002:804::200e
Connecting to encrypted-tbn0.gstatic.com|216.58.220.206|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2654 (2.6K) [image/jpeg]
Saving to: ‘images?q=tbn:ANd9GcT7S5ICg0CFXhBnCraxvE00bujuyKEBNX0oseeqbymdVA3lmO_fp3oIFg’

images?q=tbn:ANd9Gc 100%[===================>]   2.59K  --.-KB/s    in 0s      

2017-03-20 18:56:20 (19.5 MB/s) - ‘images?q=tbn:ANd9GcT7S5ICg0CFXhBnCraxvE00bujuyKEBNX0oseeqbymdVA3lmO_fp3oIFg’ saved [2654/2654]

--2017-03-20 18:56:20--  https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ18oRd6hbjpwTzu4h6uKz1sZ8Q5pFFQr-jwi5MtZQLUXEQ8NWstiyJbAg
Reusing existing connection to encrypted-tbn0.gstatic.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 3049 (3.

### Creating training dataset

In [10]:
wheat_paths = []
for fl in os.listdir('../images/wheat/'):
    path = '../images/wheat/' + fl
    wheat_paths.append(path)

In [11]:
barley_paths = []
for fl in os.listdir('../images/barley/'):
    path = '../images/barley/' + fl
    barley_paths.append(path)

In [23]:
wx, wy = set(), set()
for path in wheat_paths:
    x, y, _  = imread(path).shape
    wx.add(x); wy.add(y)

In [24]:
for path in barley_paths:
    x, y, _  = imread(path).shape
    wx.add(x); wy.add(y)
wx = list(sorted(wx))
wy = list(sorted(wy))
wx[0], wy[0]

(55, 72)

In [25]:
images, labels = [], []
for path in barley_paths:
    image = imresize(imread(path), (wx[0], wy[0]))
    images.append(image)
    labels.append('barley')
for path in wheat_paths:
    image = imresize(imread(path), (wx[0], wy[0]))
    images.append(image)
    labels.append('wheat')

In [26]:
X, y = [i.flatten() for i in images], labels

In [37]:
cf = CascadeForest(n_estimators=1000)

In [34]:
cf.fit(X, y)

CascadeForest(cv=3, n_estimators=1000,
       scoring=<function accuracy_score at 0xa7975194>, tolerance=0.01,
       validation_fraction=0.8)

In [36]:
cf.scores_

[0.7441860465116279]